<a href="https://colab.research.google.com/github/Divyangna05/Repo1/blob/main/ESG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPortfolioOpt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
from pypfopt.efficient_frontier import EfficientFrontier # import after installing
from pypfopt import risk_models, expected_returns

In [ ]:
esg_data = pd.read_csv('/content/data.csv')
print(esg_data.head())

  ticker                           name currency  \
0    dis                 Walt Disney Co      USD   
1     gm              General Motors Co      USD   
2    gww                WW Grainger Inc      USD   
3    mhk          Mohawk Industries Inc      USD   
4    lyv  Live Nation Entertainment Inc      USD   

                        exchange                            industry  \
0  NEW YORK STOCK EXCHANGE, INC.                               Media   
1  NEW YORK STOCK EXCHANGE, INC.                         Automobiles   
2  NEW YORK STOCK EXCHANGE, INC.  Trading Companies and Distributors   
3  NEW YORK STOCK EXCHANGE, INC.                   Consumer products   
4  NEW YORK STOCK EXCHANGE, INC.                               Media   

                                                logo  \
0  https://static.finnhub.io/logo/ef50b4a2b263c84...   
1  https://static.finnhub.io/logo/9253db78-80c9-1...   
2  https://static.finnhub.io/logo/f153dcda-80eb-1...   
3  https://static.finnhub.io/l

In [ ]:
tickers = esg_data['ticker'].tolist()
def fetch_data(ticker):
   try:
        return yf.download(ticker, start="2020-01-01", end="2023-01-01")
   except KeyError:
        print(f"Data not found for ticker: {ticker}")
        return None  # Return None if data is not found
stock_data = {ticker: fetch_data(ticker) for ticker in tickers}
if 'AAPL' in stock_data:
    print(stock_data['AAPL'].head())
else:
    print("AAPL data not found.")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

AAPL data not found.


In [ ]:
def calculate_esg_score(row):
    # Check if the column names exist before accessing them
    if 'Environmental' in row and 'Social' in row and 'Governance' in row:
        return row['Environmental'] * 0.4 + row['Social'] * 0.3 + row['Governance'] * 0.3
    else:
        print("Warning: Missing ESG columns in row:", row)
        return None  # Or a default value if needed

esg_data['ESG_Score'] = esg_data.apply(calculate_esg_score, axis=1)
print(esg_data.head())

Streaming output truncated to the last 5000 lines.
name                                                          Fastenal Co
currency                                                              USD
exchange                                       NASDAQ NMS - GLOBAL MARKET
industry                               Trading Companies and Distributors
logo                    https://static.finnhub.io/logo/d8a85892-80df-1...
weburl                                          https://www.fastenal.com/
environment_grade                                                       B
environment_level                                                  Medium
social_grade                                                           BB
social_level                                                       Medium
governance_grade                                                        B
governance_level                                                   Medium
environment_score                                            

In [ ]:
def calculate_average_return(ticker_data):
   ticker_data['Return'] = ticker_data['Adj Close'].pct_change()
    return ticker_data['Return'].mean()
financial_metrics = {ticker: calculate_average_return(data) for ticker, data in stock_data.items()} # Calculate only for tickers with data
# Check if 'AAPL' is in the dictionary before printing
if 'AAPL' in financial_metrics:
    print(financial_metrics['AAPL'])
else:
    print("'AAPL' not found in financial metrics. Data might be missing.")

IndentationError: unexpected indent (<ipython-input-17-bb011a485d3c>, line 3)

In [ ]:
returns = pd.DataFrame({ticker: data['Adj Close'].pct_change() for ticker, data in stock_data.items()})
mu = expected_returns.mean_historical_return(returns)
S = risk_models.sample_cov(returns)
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)


In [ ]:
def plot_portfolio(weights):
    df = pd.DataFrame(list(weights.items()), columns=['Ticker', 'Weight'])
    fig = px.pie(df, names='Ticker', values='Weight', title='Portfolio Allocation')
    return fig
fig = plot_portfolio(cleaned_weights)
fig.show()